### Etymology

In [1]:
import pandas as pd
import numpy as np
import glob
import nltk
from nltk.stem import WordNetLemmatizer
from urllib.request import urlopen
from bs4 import BeautifulSoup
from json2html import *
import re
import requests
from urllib.request import Request, urlopen
from urllib import parse
#nltk.download('wordnet')
#nltk.download('omw-1.4')

In [2]:
lang_folder = "Turkish"

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Result/Word/Merge/Word_Merge2.csv")
df_word = df_word.head(10000)
df_word

In [ ]:
#word_lemma = []
#for word in df_word["word"]:
#    word_lemma.append(WordNetLemmatizer().lemmatize(word))
#    word_unique = set(word_lemma)

In [ ]:
#len(word_lemma)

In [ ]:
#len(word_unique)

In [ ]:
word_list = df_word["word"].to_list()

In [ ]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://etymologeek.com/search/all/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_word = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        a_href = word.findAll('td', attrs={"data-th":"Dic. entry"})
        iter_values = zip(dict_word, lang, definition, a_href)
        for a, b, c, d in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()  # string
            var3 = c.get_text()  # string
            var4 = d.find('a').get('href')
            var5 = f"https://etymologeek.com{var4}"
            result_list.append([i, var1, var2, var3, var5])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"dict_entry", 2:"language", 3:"definition", 4:"href"}, inplace=True)
df_result

In [ ]:
df_result.to_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_Result.xlsx", index=False)

#### Scrapping Additional Info

In [ ]:
df_main_word = pd.read_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_Result.xlsx")
df_main_word

In [ ]:
df_main_word.search_word.nunique()

In [ ]:
df_main_word.language.nunique()

In [ ]:
search_word = df_main_word["search_word"].to_list()
dict_entry_main = df_main_word["dict_entry"].to_list()
language_main = df_main_word["language"].to_list()
definition_main = df_main_word["definition"].to_list()	
href_url = df_main_word["href"].to_list()
main_iter_values = zip(search_word, dict_entry_main, language_main, definition_main, href_url)
result_list2 = []
for search, dict_ent, lang_main, def_main, href in main_iter_values:
    try:
        response = requests.get(f"{href}")
        soup = BeautifulSoup(response.content, 'lxml')
        word = soup.find('tbody', attrs={"id":"tb"})
        dict_entry = word.findAll('td', attrs={"data-th":"Dic. entry"})
        lang = word.findAll('td', attrs={"data-th":"Language"})
        definition = word.findAll('td', attrs={"data-th":"Definition"})
        iter_values = zip(dict_entry, lang, definition)
        for a, b, c in iter_values:
            var1 = a.get_text()
            var2 = b.get_text()
            var3 = c.get_text()
            result_list2.append([search, dict_ent, lang_main, def_main, var1, var2, var3])
    except:
        pass

df_result2 = pd.DataFrame(result_list2)
df_result2.rename(columns={0:"search_word", 1:"dict_entry_main", 2:"language_main", 3:"definition_main", 4:"dict_entry", 5:"language", 6:"definition"}, inplace=True)
df_result2

In [ ]:
df_result2.language_main = df_result2.language_main.apply(lambda x: x.strip())
df_result2.language = df_result2.language.apply(lambda x: x.strip())

In [ ]:
df_result2.to_excel(f"{lang_folder.capitalize()}_Etymologeek_Main_And_Additional_Result.xlsx", index=False)

#### Github Data Process 

In [3]:
df_github = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology.csv")
df_github

,term_id,lang,term,reltype,related_term_id,related_lang,related_term,position,group_tag,parent_tag,parent_position
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,group_derived_root,NaN,NaN,NaN,0,0IIU6TlkSA2D7dg1PfxGOw,NaN,NaN
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,0.0
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,0IIU6TlkSA2D7dg1PfxGOw,1.0
3,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure,0,NaN,NaN,NaN
4,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3884332,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,AUIYyyL0V_2rTzbDwWeVig,Danish,Bøll,0,NaN,VfBh8FOWR_muVFpFl7EOvw,0.0
3884333,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,_zZUp3xTUyyi8TrPVMfOUw,German,Böll,0,NaN,VfBh8FOWR_muVFpFl7EOvw,1.0
3884334,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,QLqQUWCoUu6hQGr40QVQyw,South Levantine Arabic,بـ,0,NaN,NaN,NaN
3884335,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,sVDOI8DxUwq2DD03Y3HGdw,South Levantine Arabic,الميّة,1,NaN,NaN,NaN


##### Select 1

In [4]:
df_select = df_github.loc[:,["lang","term","reltype","related_lang","related_term"]]
df_select

,lang,term,reltype,related_lang,related_term
0,English,thesaurus,group_derived_root,NaN,NaN
1,English,thesaurus,borrowed_from,Latin,thēsaurus
2,English,thesaurus,derived_from,Ancient Greek,θησαυρός
3,English,thesaurus,doublet_with,English,treasure
4,Latin,thesaurus,derived_from,Ancient Greek,θησαυρός
...,...,...,...,...,...
3884332,English,Boell,borrowed_from,Danish,Bøll
3884333,English,Boell,borrowed_from,German,Böll
3884334,South Levantine Arabic,بالمية,has_affix,South Levantine Arabic,بـ
3884335,South Levantine Arabic,بالمية,has_affix,South Levantine Arabic,الميّة


In [5]:
df_select.isnull().sum()

lang                 0
term                53
reltype              0
related_lang    381344
related_term    381412
dtype: int64

In [6]:
df_select.drop_duplicates(inplace=True)
df_select.dropna(inplace=True)
df_select.reset_index(drop=True, inplace=True)
df_select

,lang,term,reltype,related_lang,related_term
0,English,thesaurus,borrowed_from,Latin,thēsaurus
1,English,thesaurus,derived_from,Ancient Greek,θησαυρός
2,English,thesaurus,doublet_with,English,treasure
3,Latin,thesaurus,derived_from,Ancient Greek,θησαυρός
4,English,portmanteau,has_root,Proto-Indo-European,*per- (fare)
...,...,...,...,...,...
3484911,English,Boell,borrowed_from,Danish,Bøll
3484912,English,Boell,borrowed_from,German,Böll
3484913,South Levantine Arabic,بالمية,has_affix,South Levantine Arabic,بـ
3484914,South Levantine Arabic,بالمية,has_affix,South Levantine Arabic,الميّة


In [ ]:
df_select.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv", index=False)

##### Select 2

In [7]:
df_select2 = df_github.loc[:,["term_id","lang","term","reltype","related_term_id","related_lang","related_term"]]
df_select2

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,group_derived_root,NaN,NaN,NaN
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
3,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure
4,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
...,...,...,...,...,...,...,...
3884332,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,AUIYyyL0V_2rTzbDwWeVig,Danish,Bøll
3884333,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,_zZUp3xTUyyi8TrPVMfOUw,German,Böll
3884334,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,QLqQUWCoUu6hQGr40QVQyw,South Levantine Arabic,بـ
3884335,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,sVDOI8DxUwq2DD03Y3HGdw,South Levantine Arabic,الميّة


In [8]:
df_select2.isnull().sum()

term_id                 0
lang                    0
term                   53
reltype                 0
related_term_id    381344
related_lang       381344
related_term       381412
dtype: int64

In [9]:
df_select2.drop_duplicates(inplace=True)
df_select2.dropna(inplace=True)
df_select2.reset_index(drop=True, inplace=True)
df_select2

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure
3,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
4,BuO_dtS5XKenvDCLQimf5w,English,portmanteau,has_root,FWM_rWmKWoGxUEYc9oq8JQ,Proto-Indo-European,*per- (fare)
...,...,...,...,...,...,...,...
3484911,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,AUIYyyL0V_2rTzbDwWeVig,Danish,Bøll
3484912,ttDtihHdV4aL8aDWZpIgnQ,English,Boell,borrowed_from,_zZUp3xTUyyi8TrPVMfOUw,German,Böll
3484913,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,QLqQUWCoUu6hQGr40QVQyw,South Levantine Arabic,بـ
3484914,s0ANGE2hXn6nen8AFLynOg,South Levantine Arabic,بالمية,has_affix,sVDOI8DxUwq2DD03Y3HGdw,South Levantine Arabic,الميّة


In [ ]:
df_select2.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean2.csv", index=False)

##### Sample Process

In [ ]:
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv")
df_clean = df_clean.head(150000)
df_clean

In [ ]:
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.capitalize())
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.strip())
df_clean["term"] = df_clean["term"].apply(lambda x: x.lower())
df_clean["term"] = df_clean["term"].apply(lambda x: x.strip())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.capitalize())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.strip())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.lower())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.strip()) 

In [ ]:
result_list = []
for i in range(len(df_clean)):
    lang_11 = df_clean.loc[i,"lang"]
    term_11 = df_clean.loc[i,"term"]
    reltype_11	= df_clean.loc[i,"reltype"]
    lang_12 = df_clean.loc[i,"related_lang"]
    term_12 = df_clean.loc[i,"related_term"]
    
    try:
        df_var = df_clean[(df_clean["term"] == f"{term_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):                    
            lang_21 = df_var.loc[j,"lang"]
            term_21 = df_var.loc[j,"term"]
            #reltype_21	= df_var.loc[j,"reltype"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if ((f"{lang_21}" == f"{lang_12}") & (f"{term_21}" == f"{term_12}")):
                result_list.append([lang_11,term_11,reltype_11,lang_22,term_22])
            else:
                result_list.append([lang_11,term_11,reltype_11,lang_12,term_12])
            
            j+=1
    except:
        result_list.append([lang_11,term_11,reltype_11,lang_12,term_12])

In [ ]:
df_result = pd.DataFrame(result_list, columns=["lang","term","reltype","related_lang","related_term"])
df_result.drop_duplicates(inplace=True)
df_result.reset_index(drop=True,inplace=True)
df_result

In [ ]:
#df_result.to_csv("etymology_cross_check_sample.csv", index=False)

##### Multi Process

In [10]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [11]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [12]:
# 1-Way
df_clean = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean.csv")
df_clean = df_clean.head(150000)
df_clean

,lang,term,reltype,related_lang,related_term
0,English,thesaurus,borrowed_from,Latin,thēsaurus
1,English,thesaurus,derived_from,Ancient Greek,θησαυρός
2,English,thesaurus,doublet_with,English,treasure
3,Latin,thesaurus,derived_from,Ancient Greek,θησαυρός
4,English,portmanteau,has_root,Proto-Indo-European,*per- (fare)
...,...,...,...,...,...
149995,English,architect,etymologically_related_to,Ancient Greek,ἀρχι-
149996,English,architect,etymologically_related_to,Ancient Greek,τέκτων
149997,Dutch,architect,borrowed_from,Middle French,architecte
149998,Dutch,architect,derived_from,Latin,architectus


In [13]:
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.capitalize())
df_clean["lang"] = df_clean["lang"].apply(lambda x: x.strip())
df_clean["term"] = df_clean["term"].apply(lambda x: x.lower())
df_clean["term"] = df_clean["term"].apply(lambda x: x.strip())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.capitalize())
df_clean["related_lang"] = df_clean["related_lang"].apply(lambda x: x.strip())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.lower())
df_clean["related_term"] = df_clean["related_term"].apply(lambda x: x.strip()) 

In [14]:
d_list = range(len(df_clean))

In [15]:
resultlist = []

In [16]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def cross_check(i):
    lang_11 = df_clean.loc[i,"lang"]
    term_11 = df_clean.loc[i,"term"]
    reltype_11	= df_clean.loc[i,"reltype"]
    lang_12 = df_clean.loc[i,"related_lang"]
    term_12 = df_clean.loc[i,"related_term"]
    
    try:
        df_var = df_clean[(df_clean["term"] == f"{term_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):                    
            lang_21 = df_var.loc[j,"lang"]
            term_21 = df_var.loc[j,"term"]
            reltype_21	= df_var.loc[j,"reltype"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if ((f"{lang_21}" == f"{lang_12}") & (f"{term_21}" == f"{term_12}")):
                resultlist.append([lang_11,term_11,reltype_21,lang_22,term_22])
            else:
                resultlist.append([lang_11,term_11,reltype_11,lang_12,term_12])
            
            j+=1
    except:
        resultlist.append([lang_11,term_11,reltype_11,lang_12,term_12])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(cross_check, d_list) # string_word liste

In [17]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list, columns=["lang","term","reltype","related_lang","related_term"])
df_result.drop_duplicates(inplace=True)
df_result.reset_index(drop=True,inplace=True)
df_result

,lang,term,reltype,related_lang,related_term
0,English,squid,has_root,Proto-indo-european,*gʷeyh₃-
1,English,squid,inherited_from,Middle english,quik
2,English,theirs,derived_from,Old norse,þeirra
3,English,theirs,cognate_of,Old english,heora
4,English,thesaurus,inherited_from,Middle english,tresour
...,...,...,...,...,...
237850,English,wordbook,cognate_of,Norwegian bokmål,ordbok
237851,English,wordbook,cognate_of,Norwegian nynorsk,ordbok
237852,English,wordbook,cognate_of,Volapük,vödabuk
237853,English,wordbook,compound_of,Danish,ord


In [ ]:
#df_result.to_csv("etymology_cross_check.csv", index=False)

In [ ]:
#df_result.to_excel("etymology_cross_check.xlsx", index=False)

In [18]:
# 2-Way (Master)
df_clean2 = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_clean2.csv")
df_clean2 = df_clean2.head(150000)
df_clean2

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,borrowed_from,fhl-sdgwXaWvYD8CgaKXpQ,Latin,thēsaurus
1,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
2,8_rYLlLyVOKJlGFPZIvpVA,English,thesaurus,doublet_with,sqiifJX3XIOQ0vZ0trzRaA,English,treasure
3,iwDzanYmWxaUKbOlUy2yyQ,Latin,thesaurus,derived_from,7Rf1835bWYmlH9VnyYVbpg,Ancient Greek,θησαυρός
4,BuO_dtS5XKenvDCLQimf5w,English,portmanteau,has_root,FWM_rWmKWoGxUEYc9oq8JQ,Proto-Indo-European,*per- (fare)
...,...,...,...,...,...,...,...
149995,fsUAmuRuV9OyTjbBF928Yg,English,architect,etymologically_related_to,NfL4GpkTX8uegarj6y4cig,Ancient Greek,ἀρχι-
149996,fsUAmuRuV9OyTjbBF928Yg,English,architect,etymologically_related_to,UlOg8reuXuaoaYDEInocqA,Ancient Greek,τέκτων
149997,7ZnAXsdeX3aRtVH_GgA-pw,Dutch,architect,borrowed_from,EpKAbGeRVTGuIgvHK_uUwQ,Middle French,architecte
149998,7ZnAXsdeX3aRtVH_GgA-pw,Dutch,architect,derived_from,S80mxnycUK2T-0auJJ1gSQ,Latin,architectus


In [19]:
df_clean2["lang"] = df_clean2["lang"].apply(lambda x: x.capitalize())
df_clean2["lang"] = df_clean2["lang"].apply(lambda x: x.strip())
df_clean2["term_id"] = df_clean2["term_id"].apply(lambda x: x.strip())
df_clean2["related_lang"] = df_clean2["related_lang"].apply(lambda x: x.capitalize())
df_clean2["related_lang"] = df_clean2["related_lang"].apply(lambda x: x.strip())
df_clean2["related_term_id"] = df_clean2["related_term_id"].apply(lambda x: x.strip()) 

In [20]:
d_list2 = range(len(df_clean2))

In [21]:
resultlist2 = []

In [22]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def cross_check(i):
    term_id_11 = df_clean2.loc[i,"term_id"]
    lang_11 = df_clean2.loc[i,"lang"]
    term_11 = df_clean2.loc[i,"term"]
    reltype_11	= df_clean2.loc[i,"reltype"]
    term_id_12 = df_clean2.loc[i,"related_term_id"]
    lang_12 = df_clean2.loc[i,"related_lang"]
    term_12 = df_clean2.loc[i,"related_term"]
    
    try:
        df_var = df_clean2[(df_clean2["term_id"] == f"{term_id_12}")]
        df_var.reset_index(drop=True, inplace=True)
        j = 0
        while j < len(df_var):
            term_id_21 = df_var.loc[j,"term_id"]
            #lang_21 = df_var.loc[j,"lang"]
            #term_21 = df_var.loc[j,"term"]
            reltype_21	= df_var.loc[j,"reltype"]
            term_id_22 = df_var.loc[j,"related_term_id"]
            lang_22 = df_var.loc[j,"related_lang"]
            term_22 = df_var.loc[j,"related_term"]
            if (f"{term_id_21}" == f"{term_id_12}"):
                resultlist2.append([term_id_11,lang_11,term_11,reltype_21,term_id_22,lang_22,term_22])
            else:
                resultlist2.append([term_id_11,lang_11,term_11,reltype_11,term_id_12,lang_12,term_12])
            
            j+=1
    except:
        resultlist2.append([term_id_11,lang_11,term_11,reltype_11,term_id_12,lang_12,term_12])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(cross_check, d_list2) # string_word liste

In [23]:
result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=["term_id","lang","term","reltype","related_term_id","related_lang","related_term"])
df_result2.drop_duplicates(inplace=True)
df_result2.reset_index(drop=True,inplace=True)
df_result2

,term_id,lang,term,reltype,related_term_id,related_lang,related_term
0,HloBljqTXuuM6uyqXIVsfA,English,theirs,derived_from,GjZHhrFXV0CcXb2QVaChMA,Old norse,þeirra
1,HloBljqTXuuM6uyqXIVsfA,English,theirs,cognate_of,uMqcCj_wXQWphv-G04dgpA,Old english,heora
2,KZGr0B-bUeS8B14e3Ppt_w,English,squid,has_root,p-IXotBxXjaZeDUhzoRNNg,Proto-indo-european,*gʷeyh₃-
3,KZGr0B-bUeS8B14e3Ppt_w,English,squid,inherited_from,MlbsJDjHWwiImf9DP9DZoQ,Middle english,quik
4,KZGr0B-bUeS8B14e3Ppt_w,English,squid,etymologically_related_to,dF2TvgihUKKgSk8IyLRyUg,Middle english,quic
...,...,...,...,...,...,...,...
206060,81Q0H906Wpajp0il1_cjoQ,English,wordbook,cognate_of,i_v5cwSPU8uZB32kfI0Sqg,Norwegian bokmål,ordbok
206061,81Q0H906Wpajp0il1_cjoQ,English,wordbook,cognate_of,aU3nARJ8VY6OiuXfeKcCqA,Norwegian nynorsk,ordbok
206062,81Q0H906Wpajp0il1_cjoQ,English,wordbook,cognate_of,DbPMRT9eVyeXKcF4cDh6YQ,Volapük,vödabuk
206063,81Q0H906Wpajp0il1_cjoQ,English,wordbook,compound_of,HO_E1OrrWqmIo9ieMF-oYQ,Danish,ord


In [ ]:
df_result2.to_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/etymology_cross_check2.csv", index=False)

#### Etimoloji Sözlük

In [40]:
lang_folder = "Turkish"
start_num = 0
end_num = 40000

In [41]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
df_word = df_word.iloc[start_num:end_num,]
df_word

,word,frequency
0,bir,18835735
1,fikret,12101580
2,bu,11062659
3,ne,8025880
4,ve,7766036
...,...,...
39995,ekipmanlar,974
39996,şanslılar,974
39997,aksanıyla,974
39998,yargılanacak,974


In [15]:
word_list = df_word["word"].tolist()
#word_list

In [16]:
result_list = []
#for i in word_unique:
for i in word_list:
    try:
        response = requests.get(f"https://www.etimolojiturkce.com/kelime/{i}")
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find('div', attrs={"class":"holder"})
        data_main = data.find('div', attrs={"class":"main"})
        word = data_main.find('h1').get_text() 
        all_paraf = data_main.find_all('p')        
        koken = all_paraf[3].get_text()
        span = all_paraf[3].span
        all_b = span.find_all('b')
        lang = all_b[0].get_text()
        lang_root = all_b[-1].get_text()

        result_list.append([i, word, lang, koken, lang_root])    
                
        #iter_values = zip(word3, koken2)
        #for a, b in iter_values:
        #    var1 = a  # span
        #    var2 = b  # string
        #   
        #    result_list.append([i, var1, var2])
    except:
        pass

df_result = pd.DataFrame(result_list)
df_result.rename(columns={0:"search_word", 1:"word", 2:"lang",3:"koken",4:"lang_root"}, inplace=True)
df_result

,search_word,word,lang,koken,lang_root
0,yürek,Yürek,Ana Türkçe,Ana Türkçe ? sözcüğünden +Ak3 sonekiyle türet...,Ana Türkçe
1,torba,Torba,Farsça,Farsça aynı anlama gelen tōbre توبره z sözcüğ...,Orta Farsça (Pehlevice veya Partça)
2,hoşnut,Hoşnut,Farsça ve Orta Farsça (Pehlevice veya Partça),Farsça ve Orta Farsça (Pehlevice veya Partça)...,Avesta (Zend) dilinde
3,mevzu,Mevzu,Arapça,"Arapça wḍˁ kökünden gelen mawḍūˁ موضوع z ""o...",Arapça
4,ikiz,İkiz,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *ékkiŕ ""i...",Ana Türkçe
...,...,...,...,...,...
1984,masör,Masör,Fransızca\r,"Fransızca\r masseur ""masaj yapan"" sözcüğünden...",Fransızca\r
1985,baraka,Baraka,İtalyanca\r,"İtalyanca\r baracca ""kulübe, derme çatma yapı...",İtalyanca\r
1986,oğlak,Oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
1987,boykot,Boykot,İngilizce\r,"İngilizce\r boycott ""bir mal veya hizmeti sat...",İngilizce\r


In [ ]:
df_result["word"] = df_result["word"].apply(lambda x: x.lower())

In [ ]:
def remove(x):
    try:
        x = x.replace("_x000D_","")
        x = x.replace("\n","")
        x = x.strip()
    except:
        x = x
    return x

In [ ]:
df_result["lang"] = df_result["lang"].apply(lambda x: remove(x))
df_result["koken"] = df_result["koken"].apply(lambda x: remove(x))
df_result["lang_root"] = df_result["lang_root"].apply(lambda x: remove(x))

In [17]:
df_result.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/{lang_folder.capitalize()}_{start_num}-{end_num}_Word_Etimoloji_Result.xlsx", index=False)

In [42]:
df1 = pd.read_excel("Turkish_0-10000_Word_Etimoloji_Result.xlsx")
df1

,search_word,word,lang,koken,lang_root
0,bir,Bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,Ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" f...",Arapça
2,için,İçin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğün...",Eski Türkçe
3,çok,Çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""ba...",Ana Türkçe
4,ama,Ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağ...",Arapça
...,...,...,...,...,...
1712,ebeveyn,Ebeveyn,Arapça,"Arapça ˀabawayn أبوين z ""iki babalar, ana bab...",Arapça
1713,yığın,Yığın,Eski Türkçe,Eski Türkçe yıg- fiilinden +In sonekiyle türe...,Eski Türkçe
1714,dadı,Dadı,Farsça,"Farsça dādū دادو z ""halayık, dadı"" sözcüğünde...",Farsça
1715,ender,Ender,Arapça,"Arapça ndr kökünden gelen andar أندر z ""daha ...",Arapça


In [43]:
df2 = pd.read_excel("Turkish_10000-40000_Word_Etimoloji_Result.xlsx")
df2

,search_word,word,lang,koken,lang_root
0,yürek,Yürek,Ana Türkçe,Ana Türkçe ? sözcüğünden +Ak3 sonekiyle türet...,Ana Türkçe
1,torba,Torba,Farsça,Farsça aynı anlama gelen tōbre توبره z sözcüğ...,Orta Farsça (Pehlevice veya Partça)
2,hoşnut,Hoşnut,Farsça ve Orta Farsça (Pehlevice veya Partça),Farsça ve Orta Farsça (Pehlevice veya Partça)...,Avesta (Zend) dilinde
3,mevzu,Mevzu,Arapça,"Arapça wḍˁ kökünden gelen mawḍūˁ موضوع z ""o...",Arapça
4,ikiz,İkiz,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *ékkiŕ ""i...",Ana Türkçe
...,...,...,...,...,...
1984,masör,Masör,Fransızca_x000D_,"Fransızca_x000D_ masseur ""masaj yapan"" sözcüğ...",Fransızca_x000D_
1985,baraka,Baraka,İtalyanca_x000D_,"İtalyanca_x000D_ baracca ""kulübe, derme çatma...",İtalyanca_x000D_
1986,oğlak,Oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
1987,boykot,Boykot,İngilizce_x000D_,"İngilizce_x000D_ boycott ""bir mal veya hizmet...",İngilizce_x000D_


In [44]:
df1_df2 = pd.concat([df1,df2], axis=0)
df1_df2.reset_index(drop=True, inplace=True)
df1_df2 

,search_word,word,lang,koken,lang_root
0,bir,Bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,Ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" f...",Arapça
2,için,İçin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğün...",Eski Türkçe
3,çok,Çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""ba...",Ana Türkçe
4,ama,Ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağ...",Arapça
...,...,...,...,...,...
3701,masör,Masör,Fransızca_x000D_,"Fransızca_x000D_ masseur ""masaj yapan"" sözcüğ...",Fransızca_x000D_
3702,baraka,Baraka,İtalyanca_x000D_,"İtalyanca_x000D_ baracca ""kulübe, derme çatma...",İtalyanca_x000D_
3703,oğlak,Oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
3704,boykot,Boykot,İngilizce_x000D_,"İngilizce_x000D_ boycott ""bir mal veya hizmet...",İngilizce_x000D_


In [45]:
df1_df2["word"] = df1_df2["word"].apply(lambda x: x.lower())

In [46]:
df1_df2

,search_word,word,lang,koken,lang_root
0,bir,bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" f...",Arapça
2,için,i̇çin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğün...",Eski Türkçe
3,çok,çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""ba...",Ana Türkçe
4,ama,ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağ...",Arapça
...,...,...,...,...,...
3701,masör,masör,Fransızca_x000D_,"Fransızca_x000D_ masseur ""masaj yapan"" sözcüğ...",Fransızca_x000D_
3702,baraka,baraka,İtalyanca_x000D_,"İtalyanca_x000D_ baracca ""kulübe, derme çatma...",İtalyanca_x000D_
3703,oğlak,oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
3704,boykot,boykot,İngilizce_x000D_,"İngilizce_x000D_ boycott ""bir mal veya hizmet...",İngilizce_x000D_


In [47]:
df1_df2.lang.unique()

array(['Eski Türkçe', 'Arapça', 'Ana Türkçe', 'Farsça',
       'Farsça ve Orta Farsça (Pehlevice veya Partça)',
       'Arapça ve Farsça', 'Türkçe', 'Türkiye Türkçesinde',
       'Eski Batı Türkçesinde', 'Soğdca', 'Fransızca_x000D_',
       'ticari markasından', 'İngilizce_x000D_',
       'Orta Farsça (Pehlevice veya Partça)', 'Hotan Saka dilinde',
       'Venedikçe', 'Çağatay Türkçesinde', 'İtalyanca_x000D_',
       'Sırpça_x000D_', 'Fransızca ve İngilizce_x000D_', 'Yeni Türkçe',
       'Yeni Latince', 'Soğd', 'Kırgızca_x000D_', 'Orta Yunanca',
       'Yeni Yunanca', 'Moğolca_x000D_', 'Ermenice',
       'Ses yansımalı sözcüktür.', 'Bulgarca ve Sırpça_x000D_', 'Latince',
       'Çingenece (Roma)', 'Fransızca ve İtalyanca_x000D_',
       'Almanca_x000D_', 'Eski Yunanca', 'İspanyolca_x000D_', 'İt (Sic)',
       'Rusça_x000D_', 'Çince_x000D_', 'Ladino (Yahudi İspanyolcası)',
       'Proto-İranca (Ana-İranca)', 'Macarca_x000D_', 'Japonca_x000D_',
       'Sanskritçe', 'Fransızca ve İngilizc

In [48]:
df1_df2.to_excel("test.xlsx", index=False, encoding="utf8")

In [49]:
df1_df2 = pd.read_excel("test.xlsx")
df1_df2

,search_word,word,lang,koken,lang_root
0,bir,bir,Eski Türkçe,Eski Türkçe bir sözcüğünden evrilmiştir.,Eski Türkçe
1,ve,ve,Arapça,"Arapça wa و z ""ve (bağlaç), - adına (edat)"" f...",Arapça
2,için,i̇çin,Eski Türkçe,"Eski Türkçe üçün ""nedensellik edatı"" sözcüğün...",Eski Türkçe
3,çok,çok,Ana Türkçe,"Ana Türkçe yazılı örneği bulunmayan *çaw- ""ba...",Ana Türkçe
4,ama,ama,Arapça,"Arapça ammā أمّا z ""gelgelelim, maamafih (bağ...",Arapça
...,...,...,...,...,...
3701,masör,masör,Fransızca_x000D_,"Fransızca_x000D_ masseur ""masaj yapan"" sözcüğ...",Fransızca_x000D_
3702,baraka,baraka,İtalyanca_x000D_,"İtalyanca_x000D_ baracca ""kulübe, derme çatma...",İtalyanca_x000D_
3703,oğlak,oğlak,Eski Türkçe,Eski Türkçe ogul sözcüğünden +Ak sonekiyle tü...,Eski Türkçe
3704,boykot,boykot,İngilizce_x000D_,"İngilizce_x000D_ boycott ""bir mal veya hizmet...",İngilizce_x000D_


In [50]:
def remove(x):
    try:
        x = x.replace("_x000D_","")
        x = x.replace("\n","")
        x = x.strip()
    except:
        x = x
    return x

In [51]:
df1_df2["lang"] = df1_df2["lang"].apply(lambda x: remove(x))
df1_df2["koken"] = df1_df2["koken"].apply(lambda x: remove(x))
df1_df2["lang_root"] = df1_df2["lang_root"].apply(lambda x: remove(x))

In [52]:
df1_df2.lang.unique()

array(['Eski Türkçe', 'Arapça', 'Ana Türkçe', 'Farsça',
       'Farsça ve Orta Farsça (Pehlevice veya Partça)',
       'Arapça ve Farsça', 'Türkçe', 'Türkiye Türkçesinde',
       'Eski Batı Türkçesinde', 'Soğdca', 'Fransızca',
       'ticari markasından', 'İngilizce',
       'Orta Farsça (Pehlevice veya Partça)', 'Hotan Saka dilinde',
       'Venedikçe', 'Çağatay Türkçesinde', 'İtalyanca', 'Sırpça',
       'Fransızca ve İngilizce', 'Yeni Türkçe', 'Yeni Latince', 'Soğd',
       'Kırgızca', 'Orta Yunanca', 'Yeni Yunanca', 'Moğolca', 'Ermenice',
       'Ses yansımalı sözcüktür.', 'Bulgarca ve Sırpça', 'Latince',
       'Çingenece (Roma)', 'Fransızca ve İtalyanca', 'Almanca',
       'Eski Yunanca', 'İspanyolca', 'İt (Sic)', 'Rusça', 'Çince',
       'Ladino (Yahudi İspanyolcası)', 'Proto-İranca (Ana-İranca)',
       'Macarca', 'Japonca', 'Sanskritçe',
       'Fransızca ve İngilizce ve Almanca', 'Toharca', 'İbranice',
       'Yeni Yunanca ve Eski Yunanca', 'Latince ve Eski Yunanca',
       '

In [53]:
df1_df2.lang_root.unique()

array(['Eski Türkçe', 'Arapça', 'Ana Türkçe', 'Avesta (Zend) dilinde',
       'Farsça ve Orta Farsça (Pehlevice veya Partça)', 'Akatça',
       'Türkçe', 'Türkiye Türkçesinde', 'Eski Farsça', 'Farsça',
       'İbranice', 'ünlemdir.', 'Aramice/Süryanice',
       'Eski Batı Türkçesinde', 'Sumerce', 'Hintavrupa Anadilinde',
       'Sanskritçe', 'Soğdca', 'Orta Farsça (Pehlevice veya Partça)', nan,
       'İbranice ve Aramice/Süryanice', 'Latince', 'Eski Yunanca',
       'ticari markasından', 'Almanca',
       'Eski Farsça ve Avesta (Zend) dilinde', 'Eski Fransızca',
       'Germence', 'İtalyanca', 'Moğolca', 'Yeni Türkçe', 'Maya dilinde',
       'Fenike dilinde', 'Fransızca', 'Holandaca', 'Sırpça',
       'Eski Mısır dilinde', 'İngilizce', 'Eski Slavca', 'Geç Latince',
       'İt (Sic)', 'G. Amerika dillerinden Taino dilinde', 'Gaelce',
       'Ses yansımalı sözcüktür.', 'Nahuatl (Aztek) dilinde',
       'Fransızca ve İngilizce', 'Quechua (İnka) dilinde',
       'Çingenece (Roma)', 'Etrüs

In [55]:
df1_df2.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/All/{lang_folder.capitalize()}_0-40000_Word_Etimoloji_Result2.xlsx", index=False, encoding="utf8")